In [23]:
%load_ext autoreload
%autoreload 2
%load_ext blackcellmagic

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The blackcellmagic extension is already loaded. To reload it, use:
  %reload_ext blackcellmagic


In [6]:
from IPython.core.display import display, HTML

display(HTML("<style>.container { width:80% !important; }</style>"))

In [47]:
from visualize_pyphi import *
from visualize_pyphi import utils
from visualize_pyphi import compute
import pickle as pkl
from tqdm.auto import tqdm
import numpy as np

In [70]:
# ch3 system creation
net_name = "ch3_v5"

th = 1 / 4
exp = 5
mu = 1
si = 0.3
ll = 1
kk = 15
x0 = 0.7

q = 0.6
s = 0.8
l = 0.25
k = l#+.001
j = 1 - q
i = j - 0.05
b = 0.01

weights = np.array(
    [
        [s, k, 0, i, 0, 0],  # A
        [k, s, l, j, b, 0],  # B
        [0, l, s, i, 0, 0],  # C
        [b, b, b, q, 0, i],  # D
        [0, i, 0, 0, 0, 0],  # I
        [0, 0, 0, 0, 0, 0],  # O
        # A, B, C, D, I, O
    ]
)

weights += np.random.random((6,6))/100
weights = np.abs(weights)

node_labels = ["A", "B", "C", "D", "I", "O"]
mech_func = [f for f in "sssgll"]

network = ng.get_net(
    mech_func,
    weights,
    exp=exp,
    th=th,
    mu=mu,
    si=si,
    l=ll,
    k=kk,
    x0=x0,
    node_labels=node_labels,
    network_name=net_name,
    pickle_network=True,
)

state = (0, 1, 0, 1, 1, 1)
subsystem = pyphi.Subsystem(network, state, nodes=(0, 1, 2, 3))

  0%|          | 0/64 [00:00<?, ?it/s]

Network saved to: ch3_v5


In [71]:
ces = pyphi.compute.ces(subsystem,parallel=True)

In [80]:
len(ces)

30

In [72]:
sepces = utils.flatten([[d.cause,d.effect] for d in ces])
ces = sepces

In [73]:
all_compositional_states = compute.get_all_compositional_states(ces)
print('there are {} possible compositional states'.format(len(all_compositional_states)))

there are 12 possible compositional states


In [81]:
filtered = [compute.filter_ces(subsystem, ces, compositional_state, max_relations_k=len(ces)) for compositional_state in tqdm(all_compositional_states)]

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/64 [00:00<?, ?it/s]

  0%|          | 0/64 [00:00<?, ?it/s]

  0%|          | 0/64 [00:00<?, ?it/s]

  0%|          | 0/64 [00:00<?, ?it/s]

...

  0%|          | 0/64 [00:00<?, ?it/s]

  0%|          | 0/64 [00:00<?, ?it/s]

  0%|          | 0/64 [00:00<?, ?it/s]

  0%|          | 0/64 [00:00<?, ?it/s]

  0%|          | 0/64 [00:00<?, ?it/s]

In [82]:
a = [len(m['relations']) for f in filtered for m in f]
b = [m['big phi'] for f in filtered for m in f]
c = [len(m['ces']) for f in filtered for m in f]
min(a),max(a),min(c),max(c),min(b),max(b)

(3, 159, 6, 14, 0.0, 36.1916371709527)

In [75]:
a = [len(m['relations']) for f in filtered for m in f]
b = [m['big phi'] for f in filtered for m in f]
c = [len(m['ces']) for f in filtered for m in f]
min(a),max(a),min(c),max(c),min(b),max(b)

(3, 88, 6, 14, 0.0, 8.806497418234592)

In [76]:
CES = [m for f in filtered for m in f if m['big phi']>8.8][0]
utils.sepces2df(CES['ces'],subsystem)

,mechanism,direction,purview,state,phi
0,A,CAUSE,A,0,0.512357
1,A,EFFECT,A,0,0.649867
2,B,CAUSE,B,1,0.452130
3,B,EFFECT,B,1,0.567426
4,C,CAUSE,C,0,0.514622
5,C,EFFECT,C,0,0.652147
6,AB,CAUSE,AB,10,0.180604
7,AB,EFFECT,ABD,010,0.038629
8,BC,CAUSE,BC,01,0.175186
9,BC,EFFECT,BCD,100,0.038340


In [76]:
CES = [m for f in filtered for m in f if m['big phi']>8.8][0]
utils.sepces2df(CES['ces'],subsystem)

,mechanism,direction,purview,state,phi
0,A,CAUSE,A,0,0.512357
1,A,EFFECT,A,0,0.649867
2,B,CAUSE,B,1,0.452130
3,B,EFFECT,B,1,0.567426
4,C,CAUSE,C,0,0.514622
5,C,EFFECT,C,0,0.652147
6,AB,CAUSE,AB,10,0.180604
7,AB,EFFECT,ABD,010,0.038629
8,BC,CAUSE,BC,01,0.175186
9,BC,EFFECT,BCD,100,0.038340


In [77]:
CES['MIP']

(((0,), (1, 2, 3)), (1, 2, 3), (0,), EFFECT)

In [69]:
good_weights = weights
good_weights

array([[0.80429078, 0.25524274, 0.00307273, 0.3515403 , 0.00492981,
        0.00996336],
       [0.25475904, 0.8015509 , 0.25453847, 0.40018269, 0.01702802,
        0.0043879 ],
       [0.00997719, 0.25221898, 0.80581981, 0.35990666, 0.00266345,
        0.00728655],
       [0.01578615, 0.01329649, 0.0181051 , 0.60958749, 0.00620138,
        0.35398605],
       [0.00974372, 0.35204232, 0.00702253, 0.00453444, 0.00349476,
        0.00514265],
       [0.00515462, 0.00230007, 0.00979452, 0.00469913, 0.00642495,
        0.00897918]])